<a href="https://colab.research.google.com/github/siddhant-parmar/CMPE_255_Group_4/blob/main/Experiment/Data_Mining_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd 
import json
import csv
import os
from pandas import json_normalize 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [2]:
from google.colab import files
files.upload()

! mkdir  ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.92G/4.92G [01:36<00:00, 48.9MB/s]
100% 4.92G/4.92G [01:36<00:00, 54.9MB/s]


In [5]:
from zipfile import ZipFile
file_name = 'yelp-dataset.zip'
with ZipFile(file_name,'r') as zip:
   zip.extractall()
print('Done')

Done


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls "/content/drive/My Drive"

 Accenture			      'manoj Biodata.gdoc'
 AdmitLetterViewer.php		      'Margin Requirements.gslides'
'App for Residency.gdoc'	      'Medicines .gdoc'
 authorization.docx		      'Mhada letter.gdoc'
'bank letter with sign.pdf'	       MS
 CNN_Homework.ipynb		       mushrooms.gsheet
'Colab Notebooks'		       PART-3.ipynb
'Covid vaccine delay.gdoc'	      'Ranimaa letterhead.gdoc'
 Demo_7_XGBoost.ipynb		      'resume(ajay)_compressed-converted.gdoc'
'DP PICS'			       Resume.gdoc
'ETS PRACTICE TSET 1.pdf'	      'Santa Clara I 20 request form.pdf'
'financial doc.pdf'		       Scanned
 FT_CV_AAK.docx			      'Single pics'
'Fund Main MCQ 11 AM 160 (1).gsheet'  'sjsu finance.pdf'
'Fund Main MCQ 11 AM 160.gsheet'      'SOP for Divya Khaini (1).gdoc'
'Fund MCQ Batch 7 - 2 PM.gsheet'      'SOP for Divya Khaini.docx'
'Graduate result '		      'SOP for Divya Khaini.gdoc'
'GRE Assessment Reports.pdf'	       SOP.gdoc
'GRE Magoosh Premium Videos 2017'     'Statement .gdoc'
'HOC letter head.gdoc'		      'S

In [ ]:
#review_data = pd.read_json("yelp_academic_dataset_review.json", lines=True)

Selecting only business_id and text from reviews (Feature Selection) and separating them into high risk reviews and remaining reviews in other dataframe

In [ ]:
with open("yelp_academic_dataset_review.json") as reviews:
    
    words = ["discomfort","ache","death","allergy","diarrhoea", "stomach pain", "loose motions", "loose motion", "food poisoning", "sick", "vomitting", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poisioning"]
    count = 0 
    
    highRiskReviews = pd.DataFrame(columns=['business_id', 'text'])
    restReviews = pd.DataFrame(columns=['business_id', 'text'])
    
    # Iterate through the reviews and save high risk reviews to highRiskReviews
    for index,review in enumerate(reviews):
        reviewData = json.loads(review)
        
        if any(word in reviewData['text'] for word in words) and reviewData['stars'] < 2:
            dict_rec = {'business_id': reviewData['business_id'], 'text': reviewData['text']}
            highRiskReviews = highRiskReviews.append(dict_rec, ignore_index=True)
        
        if np.random.randint(1,1000) == 1 and not any(word in reviewData['text'] for word in words):
            dict_rec = {'business_id': reviewData['business_id'], 'text': reviewData['text']}
            restReviews = restReviews.append(dict_rec, ignore_index=True)
            count += 1
        
    print(highRiskReviews.info())
    print(restReviews.info())

In [ ]:
def formatDataFromID(business_ids, highRisk = 0):
    
    X = pd.DataFrame()

    with open("yelp_academic_dataset_business.json") as businesses:
        
        for business in businesses:

            businessData = json.loads(business)

            if businessData['business_id'] in business_ids:
                if businessData['categories']:
                    if 'Food' in businessData['categories'] or 'Restaurants' in businessData['categories']:

                        # Drop columns that are unlikely to be useful features (think about dropping review count and number of stars as this will leak data)
                        columnsToDrop = ["hours", "name", "address", "city", "state", "postal_code", "latitude", "longitude", "is_open", "BusinessParking", "GoodForMeal"]
                        [businessData.pop(column, None) for column in columnsToDrop]

                        # Add in category data, represented as a string list
                        categoryColumns = businessData["categories"].split(",")
                        for category in categoryColumns:
                            businessData['category_' + category.strip()] = 1

                        # Add in attribute data, represented as a string dict
                        if businessData["attributes"]:
                            for key,value in businessData["attributes"].items():
                                if key == "Ambience":
                                    if businessData["attributes"]["Ambience"] != "None":
                                        ambienceType = businessData["attributes"].get(key)
                                        for key,value in eval(ambienceType).items():
                                            if value == 0:
                                                businessData['ambience_' + key] = 0
                                            else:
                                                businessData['ambience_' + key] = 1

                        # Assign the target variable (whether the restaurant has a high risk of food poisoning)
                        businessData['highRisk'] = highRisk

                        X = X.append(json_normalize(businessData, sep='_'), ignore_index=True)
        return X


In [ ]:
highRiskBusinesses = highRiskReviews['business_id'].tolist()
normalBusinesses = restReviews['business_id'].tolist()

X_highRisk = formatDataFromID(highRiskBusinesses, 1)
X_normal = formatDataFromID(normalBusinesses)

In [ ]:
X_highRisk.head()

In [ ]:
type(X_highRisk)

In [ ]:
print(X_highRisk.shape)
print(X_normal.shape)

Merging high Risk and normal data together to be fed into the algorithm

In [ ]:
X = pd.concat([X_highRisk, X_normal], ignore_index=True)

In [ ]:
X.shape

Encoding Business_id for easy understanding 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X['business_id'] = le.fit_transform(X['business_id'])

In [ ]:
X.shape

In [ ]:
X_clean = X.drop(['attributes_Ambience', 'categories', 'category_Restaurants', 'attributes_BusinessParking','attributes_BikeParking', 'attributes_Music'], axis=1)

cols_with_missing = [col for col in X.columns if X[col].isnull().sum() > X.shape[0]*0.9]
X_clean = X_clean.drop(cols_with_missing, axis=1)

X_clean = X_clean.fillna(0)

Only considering those businesses which have atleast 2 reviews of food poisoning

In [ ]:
counts = X_clean['business_id'].value_counts()
res = X_clean[X_clean['business_id'].isin(counts[counts > 1].index)]

In [ ]:
res.shape

Splitting the dataset into train and test set

In [ ]:
Y = res['highRisk']
X = res.iloc[:, res.columns != 'highRisk']

print(X.shape)
print(Y.shape)
print(X.head())

In [ ]:
objects = (X.dtypes == 'object')
object_cols = list(objects[objects].index)

# Drop high cardinality objects
low_cardinality_cols = [col for col in object_cols if X[col].nunique() < 10]

# Columns to be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

# Change the column data to be strings, this doesn't matter as they will be OH encoded
for col in object_cols:
    X[col] = X[col].astype(str)

X = X.drop(high_cardinality_cols, axis=1)

# OH encode the data
OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))

# Put the index back in
OH_cols.index = X.index

# Drop the columns that have been OH encoded
num_X = X.drop(low_cardinality_cols, axis=1)
OH_X = pd.concat([num_X, OH_cols], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(OH_X, Y, test_size=0.3)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators=500)
model.fit(X_train, Y_train,early_stopping_rounds=10, eval_set=[(X_test, Y_test)], verbose=False)
predictions = model.predict(X_test)
print("Mean absolute error: {:.2f}".format(mean_absolute_error(predictions, Y_test)))
print("Score for XGBoost model is {:.2f}% ".format(model.score(X_test, Y_test)*100))

In [ ]:
from sklearn import tree

DT = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DT.fit(X_train, Y_train)
y_pred = DT.predict(X_test)
print("Decision Tree")
print('Accuracy Score: {:.2f}%'.format(metrics.accuracy_score(Y_test,y_pred)*100))
print('Confusion Matrix: ',metrics.confusion_matrix(Y_test,y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, criterion='gini')
RF.fit(X_train, Y_train)
y_pred = RF.predict(X_test)
print('For Random Forest algorithm')
print('Accuracy Score: {:.2f}%'.format(metrics.accuracy_score(Y_test,y_pred)*100))
print('Confusion Matrix: ',metrics.confusion_matrix(Y_test,y_pred))